In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

# Lab 6 – APIs and Web Scraping

## DSC 80, Fall 2023

### Due Date: Monday, November 13th at 11:59 PM 


## Instructions
Welcome to the sixth lab assignment in DSC 80 this quarter!

Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook, and **you will only submit that `lab.py` file**, not this notebook!

Some additional guidelines:
- **Unlike in DSC 10, labs will have both public tests and hidden tests.** The bulk of your grade will come from your scores on hidden tests, which you will only see on Gradescope after the assignment deadline.
- **Do not change the function names in the `lab.py` file!** The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name. If you changed something you weren't supposed to, you can find the original code in the [course GitHub repository](https://github.com/dsc-courses/dsc80-2023-fa).
- Notebooks are nice for testing and experimenting with different implementations before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file, since that's all you're submitting.
- **To ensure that all of your work to be submitted is in `lab.py`, we've provided an additional uneditable notebook, called `lab-validation.ipynb`, that contains only the tests and their setup. Make sure you are able to run it top-to-bottom without error before submitting!**
- You are encouraged to write your own additional helper functions to solve the lab, as long as they also end up in `lab.py`.

**Importing code from `lab.py`**:

* Below, we import the `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.
    
***Note***: In addition to the usual imports below, we'll need the `lxml` package that isn't included in the standard `dsc80` conda environment we had you configure.

Before running the cell below for the first time, run the following line in your Terminal after activating your `dsc80` conda environment, and then restart your kernel. If you've already installed `lxml`, you should be able to run this cell without error.

```
pip install lxml
```

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from lab import *

In [4]:
import os
import pandas as pd
import numpy as np
import requests
import bs4
import lxml

## Question 1 – Practice with HTML Tags 📎

In Question 2, you'll spend plenty of time parsing HTML source code. But before you get your hands dirty trying to extract information from HTML written by other people, it is a good idea to write basic HTML code yourself. This exercise will help you better understand how the code in a `.html` file is structured.

For this question, you'll create a very basic `.html` file, named `lab06_1.html`, that satisfies the following conditions:

- It must have `<title>` and `<head>` tags.
- It must also have `<body>` tags. Within the `<body>` tags, it must have:
    - At least two headers.
    * At least three images.
        - At least one image must be a local file.
        - At least one image must be linked to online source.
        - At least one image has to have default text when it cannot be displayed.
    * At least three references (hyperlinks) to different web pages.
    * At least one table with two rows and two columns.
    

Make sure to save your file as `lab06_1.html`, and save it in the same directory as `lab.py`. **When submitting this homework to Gradescope, make sure to also upload `lab06_1.html` along with the local image that you embedded in your site.** You can upload multiple files to Gradescope at a time.
   

***Notes***:
- You can write and view basic HTML with a Jupyter Notebook, using either a Markdown cell or by using the `IPython.display.HTML` function (which takes in a string of HTML and renders it).
- If you write your HTML code within a Jupyter Notebook, you should later copy your code into a text editor and save it with the `.html` extension. You could also write your HTML in a text editor directly.
- Be sure to open your final `.html` file in a browser and make sure it looks correct on its own.

In [5]:
# Don't delete this cell!
question1()

In [6]:
grader.check("q1")

q1 results: All test cases passed!

## Question 2 – Scraping an Online Bookstore 📚

Browse through the following fake online bookstore: http://books.toscrape.com/. This website is meant for toying with scraping.

Your job is to scrape the website, collecting data on all books that have:
- **_at least_ a four-star rating**, and
- **a price _strictly_ less than £50**, and 
- **belong to specific categories** (more details below). 

You will extract the information into a DataFrame that looks like the one below.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>UPC</th>
      <th>Product Type</th>
      <th>Price (excl. tax)</th>
      <th>Price (incl. tax)</th>
      <th>Tax</th>
      <th>Availability</th>
      <th>Number of reviews</th>
      <th>Category</th>
      <th>Rating</th>
      <th>Description</th>
      <th>Title</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>e10e1e165dc8be4a</td>
      <td>Books</td>
      <td>Â£22.60</td>
      <td>Â£22.60</td>
      <td>Â£0.00</td>
      <td>In stock (19 available)</td>
      <td>0</td>
      <td>Default</td>
      <td>Four</td>
      <td>For readers of Laura Hillenbrand's Seabiscuit...</td>
      <td>The Boys in the Boat: Nine Americans...</td>
    </tr>
    <tr>
      <th>1</th>
      <td>c2e46a2ee3b4a322</td>
      <td>Books</td>
      <td>Â£25.27</td>
      <td>Â£25.27</td>
      <td>Â£0.00</td>
      <td>In stock (19 available)</td>
      <td>0</td>
      <td>Romance</td>
      <td>Five</td>
      <td>A Michelin two-star chef at twenty-eight, Violette...</td>
      <td>Chase Me (Paris Nights #2)</td>
    </tr>
    <tr>
      <th>2</th>
      <td>00bfed9e18bb36f3</td>
      <td>Books</td>
      <td>Â£34.53</td>
      <td>Â£34.53</td>
      <td>Â£0.00</td>
      <td>In stock (19 available)</td>
      <td>0</td>
      <td>Romance</td>
      <td>Five</td>
      <td>No matter how busy he keeps himself...</td>
      <td>Black Dust</td>
    </tr>
  </tbody>
</table>

To do so, implement the following functions.

<br>

#### `extract_book_links`

Complete the implementation of the function `extract_book_links`, which takes in the content of a page that contains book listings as a **string of HTML**, and returns a **list** of URLs of book-specific pages for all books with **_at least_ a four-star rating and a price _strictly_ less than £50**. 


<br>

#### `get_product_info`

Complete the implementation of the function `get_product_info`, which takes in the content of a book-specific page as a **string of HTML**, and a list `categories` of book categories. If the input book is in the list of `categories`, `get_product_info` should return a dictionary corresponding to a row in the DataFrame in the image above (where the keys are the column names and the values are the row values). If the input book is not in the list of `categories`, return `None`.


<br>

#### `scrape_books`

Finally, put everything together. Complete the implementation of the function `scrape_books`, which takes in an integer `k` and a list `categories` of book categories. `scrape_books` should use `requests` to scrape the first `k` pages of the bookstore and return a DataFrame of only the books that have 
- **_at least_ a four-star rating**, and
- **a price _strictly_ less than £50**, and
- **a category that is in the list `categories`**.

<br>

Some general guidance and tips:

- The first page of the bookstore is at http://books.toscrape.com/catalogue/page-1.html. Subsequent pages can be found by clicking the "Next" button at the bottom of the page. Look at how the URLs change each time you navigate to a new page; think about how to use f-strings (or some other string formatting technique) to generate these URLs.
- Use "inspect element" to view the source code of the pages you're trying to scrape. To find a book's category, look at the hyperlinks in the book-specific page for that book.
- **`scrape_books` should run in under 180 seconds on the entire bookstore (`k = 50`). `scrape_books` is also the only function that should make `GET` requests; the other two functions parse already-existing HTML.**
- When instantiating `bs4.BeautifulSoup` objects, use the optional argument `features='lxml'` to suppress any warnings.
- Don't worry about typecasting, i.e. it's fine if `'Number of reviews'` is not stored as type `int`. Also, don't worry if you run into encoding errors in your price columns (as the example DataFrame at the top of this cell contains).

In [44]:
extract_book_links_fp = os.path.join('data', 'products.html')
soup1 = open(extract_book_links_fp, encoding='utf-8').read()
soup = bs4.BeautifulSoup(soup1, features='html.parser')
books = soup.find_all('article', class_='product_pod')
book_links =[]
for book in books:
    p_rating = book.find('p', class_='star-rating')
    rating_classes = p_rating.get('class')
    if 'Four' not in rating_classes and 'Five' not in rating_classes:
        continue
    price = book.find('p', class_='price_color').text
    price_value = float(price.split('£')[1])
    if price_value >= 50:
        continue
    link = book.find('h3').find('a').get('href')
    book_links.append(link)

In [63]:
get_product_info_fp = os.path.join('data', 'Frankenstein.html')
proinfo = open(get_product_info_fp, encoding='utf-8').read()
soup = bs4.BeautifulSoup(proinfo)
category = soup.find('ul', class_='breadcrumb').find_all('li')[2].get_text(strip=True)
categories = ['Default']
if category in categories:
        product_info = {}
        info_tables = soup.find_all('table', class_='table table-striped')
        info_table = info_tables[0]
        rows = info_table.find_all('tr')
        for row in rows:
                key = row.find('th').get_text(strip=True)
                value = row.find('td').get_text(strip=True)
                product_info[key] = value
                product_info['Category'] = category
                product_info['Rating'] = soup.find('p', class_='star-rating').get('class')[1]
                product_info['Description'] = soup.find('div', id='product_description').find_next_sibling().get_text(strip=True)
                product_info['Title'] = soup.find('div', class_='col-sm-6 product_main').find('h1').get_text(strip=True)
else:
        None

In [68]:
all_pages = download_page(str(1))
bookLinks = extract_book_links(all_pages)
for link in bookLinks:
  book_url = f'https://books.toscrape.com/catalogue/{link}'
  request = requests.get(book_url).content
  prodInfo = get_product_info(request, ['Mystery'])
prodInfo

In [69]:
scrape_books(1, ['Mystery'])

,UPC,Category,Rating,Description,Title,Product Type,Price (excl. tax),Price (incl. tax),Tax,Availability,Number of reviews
0,e00eb4fd7b871a48,Mystery,Four,"WICKED above her hipbone, GIRL across her hear...",Sharp Objects,Books,£47.82,£47.82,£0.00,In stock (20 available),0


In [70]:
# don't delete this cell, but do run it -- it is needed for the autograder tests

# public test for extract_book_links 
extract_book_links_fp = os.path.join('data', 'products.html')
extract_book_out = extract_book_links(
    open(extract_book_links_fp, encoding='utf-8').read()
)
extract_book_url = 'scarlet-the-lunar-chronicles-2_218/index.html'

# doc tests for get product info
get_product_info_fp = os.path.join('data', 'Frankenstein.html')
get_product_info_out = get_product_info(
    open(get_product_info_fp, encoding='utf-8').read(), ['Default']
)

# public test for scrape books 
scrape_books_out = scrape_books(1, ['Mystery'])

In [73]:
scrape_books_out

,UPC,Category,Rating,Description,Title,Product Type,Price (excl. tax),Price (incl. tax),Tax,Availability,Number of reviews
0,e00eb4fd7b871a48,Mystery,Four,"WICKED above her hipbone, GIRL across her hear...",Sharp Objects,Books,£47.82,£47.82,£0.00,In stock (20 available),0


In [71]:
grader.check("q2")

q2 results: All test cases passed!

## Question 3 – API Requests 🤑

You trade stocks as a hobby. As an avid `pandas` coder, you decide to calculate statistics of your favorite stocks by pulling data from a public API. The API we will work with can be found at https://financialmodelingprep.com/developer/docs/#Stock-Historical-Price. Specifically, we will use the "**Historical Daily Prices with change and volume interval**" endpoint (search for it at the linked page).

Some relevant definitions:
- Ticker: A short code that refers to a stock. For example, Apple's ticker is AAPL and Ford's ticker is F. 
- Open: The price of a stock at the beginning of a trading day.
- Close: The price of a stock at the end of a trading day.
- Volume: The total number of shares traded in a day.
- Percent change: The difference in price with respect to the original price, as a percentage.

To make requests to the aforementioned API, you will need an API key. In order to get one, you will need to make an account at the website. Once you've signed up, you can use the API key that comes with the free plan. It has a limit of 250 requests per day, which should be more than enough. You will have to encode your API key in the URL that you make requests to; see a complete example of such a request at the right side of the [documentation](https://site.financialmodelingprep.com/developer/docs#Stock-Historical-Price).

Implement the following two functions.

#### `stock_history`

Complete the implementation of the function `stock_history`, which takes in a string `ticker` and two integers, `year` and `month`, and returns a DataFrame containing the price history for that stock in that month. Keep all of the attributes that are returned by the API.

***Notes***:
- Read the API documentation if you get stuck!
- [`pd.date_range`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html) takes in two dates and returns a sequence of all dates between the two dates, excluding the right endpoint. How might this be helpful?
- The [`requests.get`](https://docs.python-requests.org/en/master/user/quickstart/) function returns a Response object, not the data itself. Use the `json` method on the Response object to extract the relevant JSON, as we did in Lecture 15 (you don't need to `import json` to do this).
- You can instantiate a DataFrame using a sequence of dictionaries.

<br>

#### `stock_stats`

Create a function `stock_stats` that takes in a DataFrame outputted by `stock_history` and returns a **tuple** of two numbers:
1. The percent change of the stock throughout the month as a **percentage**.
2. An estimate of the total transaction volume **in billion of dollars** for that month.

Both values in the tuple should be **strings** that contain numbers rounded to two decimal places. Add a plus or minus sign in front of the percent change, and make sure that the total transaction volume string ends in a `'B'`.

**To compute the percent change**, use the opening price on the first day of the month as the starting price and the closing price on the last day of the month as the ending price.

**To compute the total transaction volume**, assume that on any given day, the average price of a share is the midpoint of the high and low price for that day.

$$ \text{Estimated Total Transaction Volume (in dollars)} = \text{Volume (number of shares traded)} \times \text{Average Price} $$

For example, suppose there are only three days in March – March 1st, March 2nd, and March 3rd.

If BYND (Beyond Meat) opens at \\$4 on March 1st and closes at \\$5 on March 3rd, its percent change for the month of March is $$\frac{\$5-\$4}{\$4} = +25.00\%$$

Suppose the high and low prices and volumes of BYND on each day are given below.
- March 1st: high \\$5, low \\$3, volume 500 million (0.5 billion)
- March 2nd: high \\$5.5, low \\$2.5, volume 1 billion
- March 3rd: high \\$5.25, low \\$4, volume 500 million (0.5 billion)

Then, the estimated total transaction volume is
$$\frac{\$5 + \$3}{2} \cdot 0.5 B + \frac{\$5.5 + \$2.5}{2} \cdot 1 B + \frac{\$5.25 + \$4}{2} \cdot 0.5 B = 8.3125B$$

In [166]:
year = 2021
month = 12
ticker = "AAPL"
start_date = f"{year}-{month:02d}-01"
end_date = f"{year}-{month+1:02d}-01" if month != 12 else f"{year+1}-01-01"
date_range = pd.date_range(start=start_date, end=end_date, closed='left')
url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?from={start_date}&to={end_date}&apikey=Tiq0MdfbMMWj2qVheY8kM0IMz8LDBcDM"
r = requests.get(url)
rr = pd.DataFrame(r.json())
historical_data = rr['historical'] if 'historical' in rr else []
df = pd.json_normalize(historical_data)
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'].isin(date_range)]

In [202]:
history = stock_history('BYND', 2019, 6)
history.head()

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
1,2019-06-28,165.30,168.80,159.5500,160.68,160.679993,7315297,7315300,-4.62,-2.79000,163.92,"June 28, 19",-0.027900
2,2019-06-27,157.31,164.79,155.4500,162.91,162.910004,5719421,5731400,5.60,3.56000,160.43,"June 27, 19",0.035600
3,2019-06-26,160.10,162.25,153.0200,160.48,160.479996,6378629,6378600,0.38,0.23735,157.99,"June 26, 19",0.002374
4,2019-06-25,138.50,150.69,138.3425,150.60,150.600006,6682929,6632500,12.10,8.74000,146.35,"June 25, 19",0.087400
5,2019-06-24,151.88,152.70,138.0000,140.99,140.990005,6538497,6538500,-10.89,-7.17000,142.69,"June 24, 19",-0.071700


In [210]:
start_price = df.iloc[-1]['open']
end_price = df.iloc[0]['close']
percent_change = ((end_price - start_price) / start_price) * 100
percent_change_str = f"{percent_change:+.2f}%"

daily_volume_billion = (df['high'] + df['low']) / 2 * df['volume'] / 1e9
total_volume_billion = daily_volume_billion.sum()
total_volume_billion_str = f"{total_volume_billion:.2f}B"

In [261]:
# don't delete this cell, but do run it -- it is needed for the autograder tests

# public test for stock_history
history = stock_history('BYND', 2019, 6)

# public test for stock_stats
stats = stock_stats(history)

In [262]:
history

,date,open,high,low,close,adjClose,volume,unadjustedVolume,change,changePercent,vwap,label,changeOverTime
1,2019-06-28,165.30,168.80,159.5500,160.68,160.679993,7315297,7315300,-4.62,-2.79000,163.92,"June 28, 19",-0.027900
2,2019-06-27,157.31,164.79,155.4500,162.91,162.910004,5719421,5731400,5.60,3.56000,160.43,"June 27, 19",0.035600
3,2019-06-26,160.10,162.25,153.0200,160.48,160.479996,6378629,6378600,0.38,0.23735,157.99,"June 26, 19",0.002374
4,2019-06-25,138.50,150.69,138.3425,150.60,150.600006,6682929,6632500,12.10,8.74000,146.35,"June 25, 19",0.087400
5,2019-06-24,151.88,152.70,138.0000,140.99,140.990005,6538497,6538500,-10.89,-7.17000,142.69,"June 24, 19",-0.071700
6,2019-06-21,153.54,161.79,150.0000,154.13,154.130005,7474586,7474600,0.59,0.38426,155.01,"June 21, 19",0.003843
7,2019-06-20,173.00,174.00,163.3000,165.17,165.169998,6660492,6660500,-7.83,-4.53000,167.58,"June 20, 19",-0.045300
8,2019-06-19,171.37,174.45,162.2500,169.28,169.279999,9451961,9452000,-2.09,-1.22000,167.61,"June 19, 19",-0.012200
9,2019-06-18,200.00,201.88,160.7000,169.89,169.889999,23966910,23966900,-30.11,-15.06000,175.95,"June 18, 19",-0.150600
10,2019-06-17,163.18,171.19,160.6111,169.96,169.960007,14626683,14626700,6.78,4.15000,166.83,"June 17, 19",0.041500


In [263]:
grader.check("q3")

q3 results: All test cases passed!

## Question 4 – Comment Threads 🧵

You regularly browse [Hacker News](https://news.ycombinator.com/) to keep up with the latest news in tech. One link to a Hacker News article is https://news.ycombinator.com/item?id=18344932. Note that this article has 18 comments and has a `storyid` of 18344932.

The problem now is that you don't have internet access on your phone during your morning commute to work, so you want to save the interesting stories' comment threads beforehand in a CSV. You find their [API documentation](https://github.com/HackerNews/API) and decide to get to work.

Complete the implementation of the function `get_comments`, which takes in a `storyid` and returns a DataFrame of all the comments below the news story. You can ignore "dead" comments(you will know them when you see them), as well as "dead" comments’ children. **Make sure the order of the comments in your DataFrame is from top to bottom just as you see on the website**. 

The DataFrame that `get_comments` returns should have 5 columns:
1. `'id'`: The unique ID of the comment.
2. `'by'`: The author of the comment.
3. `'text'`: The actual comment.
4. `'parent'`: The unique ID of the comment this comment is replying to.
5. `'time'`: When the comment was created (in `pd.Timestamp` format).

Some guidance:
1. The URL to make requests to is `'https://hacker-news.firebaseio.com/v0/item/{}.json'`, however, the `{}` should be replaced with the ID of the article or page you are trying to access. 
2. Again, do not `import json` – instead, use the `json` method on the Response object you get back.
3. Use depth-first search when traversing the comments tree. You will have to do this manually, since you cannot use Beautiful Soup (which is only for HTML documents, not JSON objects).
4. Make sure the length of your returned DataFrame is the same as value for the `'descendants'` key in the response JSON (both of which correspond to the number of comments for the story).
5. You are allowed to use loops in this function. You may also want to create at least one helper function.

<div class="alert alert-block alert-success">
    You may find <a href="https://www.youtube.com/watch?v=uOfwW-onmpc"><b>this hint video 🎥</b></a> helpful!
</div>

In [231]:
def formal_url(storyid):
    return f'https://hacker-news.firebaseio.com/v0/item/{storyid}.json'

In [232]:
def process_comment(commentid, comment_datas):
    comment_data = requests.get(formal_url(commentid)).json()
    if comment_data.get("dead"):
        return
    comment_datas.append({
        'id': comment_data['id'],
        'by': comment_data.get('by', ''),
        'text': comment_data.get('text', ''),
        'parent': comment_data.get('parent', ''),
        'time': pd.Timestamp(comment_data['time'], unit='s')
    })
    if 'kids' in comment_data:
        for child_commentid in comment_data['kids']:
            process_comment(child_commentid, comment_datas)

In [233]:
def get_comments(storyid):
    r = requests.get(formal_url(storyid)).json()
    comment_datas = []
    for commentid in r.get('kids', []):
        process_comment(commentid, comment_datas)
    comments_df = pd.DataFrame(comment_datas)
    return comments_df

In [234]:
# don't delete this cell, but do run it -- it is needed for the autograder tests
comments = get_comments(18344932)

In [236]:
grader.check("q4")

q4 results: All test cases passed!

## Congratulations! You're done with Lab 6! 🏁

As a reminder, all of the work you want to submit needs to be in `lab.py`.

To verify that all of your work is indeed in `lab.py`, and that you didn't accidentally implement a function in this notebook and not in `lab.py`, we've included another notebook in the lab folder, called `lab-validation.ipynb`. `lab-validation.ipynb` is a version of this notebook with only the `grader.check` cells and the code needed to set up the tests. 


### **Go to `lab-validation.ipynb`, and go to Kernel > Restart & Run All.** This will check if all `grader.check` test cases pass using just the code in `lab.py`.

Once you're able to pass all test cases in `lab-validation.ipynb`, including the call to `grader.check_all()` at the very bottom, then you're ready to submit your `lab.py` to Gradescope. In addition to submitting `lab.py`, you will also need to submit `lab06_1.html` and any local images necessary for rendering your `lab06_1.html` file. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()